In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, callback_context, html, dash_table
import plotly.express as px
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "password1"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.A(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
        href='https://www.snhu.edu',
        target='_blank'  # Opens the link in a new tab
    )),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div(className="radioRow",
        style={'display': 'flex'},
        children=[
            dcc.RadioItems(
                id='filter-radio',
                options=[
                    {'label': 'Water Rescue', 'value': 'wr'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': 'mwr'},
                    {'label': 'Disaster or Individual Tracking', 'value': 'dit'},
                    {'label': 'Reset', 'value': 'reset'}
                ],
                value='reset',
                labelStyle={'margin-right': '15px'}
            )
        ]
    ),
        
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.H2("CS-340 Client/Server Development Project 2 - Brian Engel")
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id', 'data'),
    Output('datatable-id', 'columns'),
    Output('datatable-id', 'selected_rows')],
    [Input('filter-radio', 'value')]
)
def update_data_table(filter_radio):
    if filter_radio == 'reset':
        # Fetch all records from the AnimalShelter module
        data = pd.DataFrame.from_records(db.read({}))
        
    elif filter_radio == 'wr':
        # Filter the data for "Water Rescue"
        data = pd.DataFrame.from_records(db.read({
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }))

    elif filter_radio == 'mwr':
        # Filter the data for "Mountain or Wilderness Rescue"
        data = pd.DataFrame.from_records(db.read({
            'breed': {'$in': ['Rottweiler', 'Siberian Husky', 'Old English Sheepdog', 'Alaskan Malamute', 'German Shepherd']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }))
        
    elif filter_radio == 'dit':
        # Filter the data for "Disaster or Individual Tracking"
        data = pd.DataFrame.from_records(db.read({
            'breed': {'$in': ['Rottweiler', 'Bloodhound', 'Golden Retriever', 'German Shepherd', 'Doberman Pinscher']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }))
    
    else:
        # Implement logic for other filter options
        # Update 'data' and 'columns' accordingly
        # For now, return an empty data and columns to prevent errors
        return [], [], []
    
    # Remove the '_id' column
    data.drop(columns=['_id'], inplace=True)

    # Define columns for the data table
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in data.columns]

    return data.to_dict('records'), columns, [0]


# Display the breeds of animal based on quantity represented in
# the data table
# In the original / reset mode there are too many breeds so i filtered it down to
# ones that were greater than 1% and grouped the rest into a other category
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):

    if viewData is None:
        return []

    df_pie = pd.DataFrame(viewData)

    # Calculate the percentage of each breed
    breed_counts = df_pie['breed'].value_counts(normalize=True) * 100

    # Filter breeds above 1%
    popular_breeds = breed_counts[breed_counts > 1].index.tolist()

    # Group breeds below 1% into 'Others'
    df_pie.loc[~df_pie['breed'].isin(popular_breeds), 'breed'] = 'Others'

    return [
        dcc.Graph(
            figure=px.pie(df_pie, names='breed', title='Preferred Animals')
        )
    ]
    
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]



# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return []
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=False)


initialized
Dash app running on http://127.0.0.1:25402/
